In [1]:
import string
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

from collections import Counter
from nltk.tokenize import TweetTokenizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

# Python script for confusion matrix creation 
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

np.set_printoptions(precision=5)
%matplotlib inline
pd.options.display.max_columns=1000

In [2]:
def updated_dict(d, **kwargs):
    updated_d = d.copy()
    updated_d.update(kwargs)
    return updated_d

def clean_func_names(df):
    df = df.copy()   
    columns = [c for c in df.columns if callable(df[c][0])] 
    for c in columns:
        funcs = []
        for train in df[c]:
            funcs.append(train.__name__)
        df[c] = funcs
    return df
    

In [3]:
# setting hyperparamaters:
SCALED = True

In [4]:
# Load data
df = pd.read_csv('SMSSpamCollection.txt', sep='\t', header=None, names=['spam', 'text'])

# set categorical values of spam to 0 or 1
df['spam'] = df['spam'] == 'spam' # makes True/False instead of "spam" and "ham"
df['spam'] = df['spam'].astype(int)  # number values instead of boolean value

# Adding new feature 'length'
L = []
for i in df.text:
    L.append(len(i))
df['length'] = L

# Add second engineered feature 'num_words'
words = df.copy()
num_words = []
for i in range(len(words.text)):
    value = words['text'][i].split(' ')
    num_words.append(len(value))
num_words
df['num_words'] = num_words

#Create sub DataFrame
sub_df = df[['text', 'length', 'num_words']]

# Split train test
X_train,  X_test, y_train, y_test =  train_test_split(sub_df, df.spam.values, test_size=0.4, random_state=42)


In [5]:
type(X_train.values)

numpy.ndarray

We don't need to scale features, as we have just one. However, in later models we use more than just this features and therefore as exercise, we do it right now as well.

In [6]:
# Use TweetTokenizer 
tknzr = TweetTokenizer()
X_train['text'] = X_train.text.apply(tknzr.tokenize)
X_test['text'] = X_test.text.apply(tknzr.tokenize)

C:\Users\milen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\milen\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
text_train = X_train.text
text_test = X_test.text

# We splitted data and therefore we need to get list of indices to iterate over if we want to create 'documents' variable
id_tr = list(text_train.index.values)
id_tr.sort()
doc_train = []
for i in id_tr:
    doc_train.append(' '.join(text_train[i]))
    
# The same for test set
id_ts = list(text_test.index.values)
id_ts.sort()

doc_test = []
for i in id_ts:
    doc_test.append(' '.join(text_test[i]))      


In [8]:
# Method .toarray() assures that we gain dense matrix
tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_text_tr = tfidf.fit_transform(doc_train).toarray()
X_text_ts = tfidf.transform(doc_test).toarray()

```python
pca = PCA(n_components=100)
pca.fit(X_text_tr)
X_text_tr_pca = pca.transform(X_text_tr)
X_text_ts_pca = pca.transform(X_text_ts)
```

In [9]:
X_train = X_train[['length', 'num_words']].values
X_test = X_test[['length', 'num_words']].values

In [10]:
# We will used MinMaxScaler, which scales values in a way that our new values will be within itnerval <0,1>.
# ATTENTION! With train set we use .fit_transform method(), with test set only .transform()!!!
if SCALED == True:
    scaler = MinMaxScaler()
    X_train_sc = scaler.fit_transform(X_train)#.reshape(-1, 1))
    X_test_sc = scaler.transform(X_test)#.reshape(-1, 1))
#else:
 #   X_train_sc = X_train.reshape(-1, 1)
     #X_test_sc = X_test.reshape(-1, 1)

C:\Users\milen\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [11]:
X_train_sc = np.concatenate((X_train_sc, X_text_tr), axis=1)
X_test_sc = np.concatenate((X_test_sc, X_text_ts), axis=1)

## Train and model

In [12]:
def train_logreg(X_train_sc, y_train, **kwargs):
    clf = LogisticRegression(random_state=0, 
                             class_weight='balanced',
                             solver=kwargs.get('solver', 'sag'), # instead of defaults you can have {} which would rise exception
                             penalty=kwargs.get('penalty', 'l2'), 
                             C=kwargs.get('C', 1.0))
    model = clf.fit(X_train_sc, y_train)
    return model 

def train_SVM(X_train_sc, y_train, **kwargs):
    clf = SVC(random_state=0, 
                             class_weight='balanced',
                             kernel=kwargs.get('kernel', 'rbf'), # instead of defaults you can have {} which would rise exception
                             coef0=kwargs.get('coef0', 0.0), 
                             C=kwargs.get('C', 1.0))
    model = clf.fit(X_train_sc, y_train)
    return model 

def train_random(X_train_sc, y_train, **kwargs):
    clf = RandomForestClassifier(random_state=0, class_weight='balanced', 
                                 bootstrap=kwargs.get('bootstrap', 'True'),
                                 n_estimators=kwargs.get('n_estimators', 100))
   
    model = clf.fit(X_train_sc, y_train)
    return model 


In [13]:
# return as pandas series, with multiple evaulation metrcis (fp, tn, fn, tp)
def eval_model(X_test_sc, y_test, X_train_sc, y_train):
    hp['test_score'] =  model.score(X_test_sc, y_test)
    hp['train_score'] = model.score(X_train_sc, y_train) 
    hp['tn'], hp['fp'], hp['fn'], hp['tp'] = confusion_matrix(y_test, model.predict(X_test_sc)).ravel()
    hp['auc score'] =   roc_auc_score(y_test, model.predict(X_test_sc))
    hp['f1_score'] = f1_score(y_test, model.predict(X_test_sc), average='weighted', labels=np.unique(model.predict(X_test_sc)))
    hp['recall'] = recall_score(y_test, model.predict(X_test_sc), average='weighted', labels=np.unique(model.predict(X_test_sc)))
    hp['precision'] = precision_score(y_test, model.predict(X_test_sc), average='weighted', labels=np.unique(model.predict(X_test_sc)))
    return hp

```python
hp['test_score'] =  eval_model(X_test_sc, y_test)
    hp['train_score'] = eval_model(X_train_sc, y_train) 
    hp['tn'], hp['fp'], hp['fn'], hp['tp'] = confusion_matrix(y_test, model.predict(X_test_sc)).ravel()
    hp['auc score'] =   roc_auc_score(y_test, model.predict(X_test_sc))
    scores.append(hp)
```

In [ ]:
scores = []
logreg_def_hyperpar = dict(train_function=train_logreg, solver='liblinear', penalty='l1', C=1.0)
SVM_def_hyperpar = dict(train_function=train_SVM, kernel='rbf', C=1.0, coef0=0.0)
RF_def_hyperpar = dict(train_function=train_random, bootstrap=True, n_estimators=1)

SVM_hyperparameters = [SVM_def_hyperpar]#, updated_dict(SVM_def_hyperpar, coef0=0.5),
                     #  updated_dict(SVM_def_hyperpar, C=0.5),
                    #   updated_dict(SVM_def_hyperpar,C=0.5, coef0=0.5),
                   #    updated_dict(SVM_def_hyperpar, C=0.1),
                   #    updated_dict(SVM_def_hyperpar, coef0=0.5),
                      # updated_dict(SVM_def_hyperpar, kernel='linear'),
                       #updated_dict(SVM_def_hyperpar,kernel='sigmoid')
                   #   ]   

RF_hyperparameters =[RF_def_hyperpar,updated_dict(RF_def_hyperpar, n_estimators=10),
                     updated_dict(RF_def_hyperpar, n_estimators=20),
                     updated_dict(RF_def_hyperpar, n_estimators=50),
                     updated_dict(RF_def_hyperpar, n_estimators=100)]
                   # ]#, updated_dict(RF_def_hyperpar,bootstrap=False),
                    #updated_dict(RF_def_hyperpar,n_estimators=1),
                    # updated_dict(RF_def_hyperpar,n_estimators=1000),
                   #  updated_dict(RF_def_hyperpar,n_estimators=50),
                  #   updated_dict(RF_def_hyperpar,n_estimators=1000, bootstrap=False)
                  #  ]
logreg_hyperparameters = [logreg_def_hyperpar]#, updated_dict(logreg_def_hyperpar, C=0.5),
            #             updated_dict(logreg_def_hyperpar, C=0.1),
                   #      updated_dict(logreg_def_hyperpar,solver='sag', penalty='l2'),
          #               updated_dict(logreg_def_hyperpar, solver='sag', penalty='l2', C=0.5),
           #              updated_dict(logreg_def_hyperpar,solver='sag', penalty='l2', C=0.1),
                  #       updated_dict(logreg_def_hyperpar,solver='newton-cg', penalty='l2'),
         #                updated_dict(logreg_def_hyperpar,solver='newton-cg', penalty='l2', C=0.5),
        #                 updated_dict(logreg_def_hyperpar, solver='newton-cg', penalty='l2', C=0.1),
       #                  updated_dict(logreg_def_hyperpar,solver='lbfgs', penalty='l2'),  
      #                   updated_dict(logreg_def_hyperpar,solver='lbfgs', penalty='l2', C=0.5),
     #                     updated_dict(logreg_def_hyperpar,solver='lbfgs', penalty='l2', C=0.1)
                     #    ]


for hyperparameters in SVM_hyperparameters:#(logreg_hyperparameters + SVM_hyperparameters + RF_hyperparameters):
    hp = logreg_def_hyperpar.copy()
    hp.update(hyperparameters)
    train = hp.get('train_function')
    print(hp)
    model = train(X_train_sc, y_train, **hp)
    hp = eval_model(X_test_sc, y_test, X_train_sc, y_train)
    scores.append(hp)
print("I am done!")

{'train_function': <function train_SVM at 0x000001E05790EEA0>, 'solver': 'liblinear', 'penalty': 'l1', 'C': 1.0, 'kernel': 'rbf', 'coef0': 0.0}


In [ ]:
clean_func_names(pd.DataFrame(scores))

In [ ]:
#df = clean_func_names(pd.DataFrame(scores))

#df = df[['train_function','precision', 'recall', 'f1_score',  'auc score', 'tn', 'fp', 'fn', 'tp', 'test_score', 
  #      'train_score', 'C', 'solver', 'kernel', 'penalty', 'bootstrap', 'coef0',  
   #    'n_estimators']]



In [ ]:
df[df['train_function']=='train_logreg'].nlargest(3, 'f1_score')

In [ ]:
df[df['train_function']=='train_SVM'].nlargest(3, 'f1_score')

In [ ]:
df[df['train_function']=='train_random'].nlargest(3,'f1_score')

In [ ]:
df.nlargest(3, 'f1_score')